# Case - Facial Beauty

Copyright © 2019 Hsu Shih-Chieh

In [2]:
%load_ext autoreload
%autoreload 2


import numpy as np
from random import randint
from datasets import load_facialbeauty
import tensorflow as tf
from sklearn.model_selection import train_test_split

from keras.applications import MobileNet, MobileNetV2,DenseNet169, NASNetLarge, InceptionResNetV2, ResNet50
from keras.applications.inception_resnet_v2 import preprocess_input
#from efficientnet import EfficientNetB0

from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, TensorBoard, ReduceLROnPlateau, LambdaCallback

import matplotlib.pyplot as plt

from datasets import load_facialbeauty
plt.rcParams['font.size'] = 9
plt.rcParams['figure.figsize'] = (9,9)





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data

In [ ]:
data = load_facialbeauty(qty=1000)
print(data.DESCR)
X_tr, X_ts, y_tr, y_ts = train_test_split(data.data, data.target, test_size=0.2, random_state=40)
X_tr, X_val, y_tr, y_val = train_test_split(X_tr, y_tr, test_size=0.2, random_state=40)


### Build Model

In [ ]:
input_shape = (350, 350, 3)
base_model=InceptionResNetV2(weights='imagenet',include_top=False, input_shape=input_shape) 
x=base_model.output
x=GlobalAveragePooling2D()(x)
preds=Dense(1)(x) #final layer with softmax activation
model=Model(inputs=base_model.input,outputs=preds)
#sgd = SGD(lr= 0.01, decay=7e-05, momentum=0.5, nesterov=True)
model.compile(optimizer='adam', loss='mse')    




### Training 

In [ ]:

imgAug = ImageDataGenerator(
    preprocessing_function=preprocess_input,
#    width_shift_range=0.001,
#    height_shift_range=0.001,
)


#imgAug.fit(X_tr)
batch=21
dgen_val = imgAug.flow(X_val, y_val, batch_size=batch)
dgen_tr = imgAug.flow(X_tr, y_tr, batch_size=batch)

early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, mode='min', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001)
#_, checkpoint_file = tempfile.mkstemp('.h5')
#print('checkpoint_file: ',checkpoint_file)
#checkpointer=ModelCheckpoint(monitor='val_loss',filepath=checkpoint_file, verbose=1, save_best_only=True, save_weights_only=False)
#tboard = tf.keras.callbacks.TensorBoard(log_dir=logdir)
callbacks = [reduce_lr, early_stop]
history = model.fit_generator(dgen_tr, steps_per_epoch=X_tr.shape[0]//batch, validation_steps=X_val.shape[0]//batch, epochs=50, validation_data=dgen_val, callbacks=callbacks) 


In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()



### Check Test Data Peroformance

In [ ]:
imgAug2 = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

dgen_ts = imgAug2.flow(X_ts, shuffle=False, batch_size=100)
preds = model.predict_generator(dgen_ts, steps=X_ts.shape[0]//100)

plt.scatter(y_ts, preds)
plt.plot(y_ts, y_ts, 'ro')


In [ ]:

nb_test_samples = X_ts.shape[0]
nb_rows, nb_cols = 5, 5
plt.figure(figsize=(10,10))
for k in range(nb_rows * nb_cols):
    i = randint(0, nb_test_samples - 1)
    pred = model.predict(np.expand_dims(preprocess_input(X_ts[i].copy()), axis=0))
    plt.subplot(nb_rows, nb_cols, k+1)
    plt.imshow(X_ts[i].astype(np.uint8))
    plt.title("p:%.2f / a:%.2f" % (pred[0][0], y_ts[i]))
    plt.axis('off')



### Tensorflow lite model

**save model to tflite format**

In [ ]:
tflite_model_file = 'tfmodel/facialbeauty.tflite'
modelname='model/facialbeauty.h5'
model.save(modelname)
converter = tf.lite.TFLiteConverter.from_keras_model_file(modelname, input_arrays=['input_4'], input_shapes={'input_4':[None,input_shape[0],input_shape[1],input_shape[2]]})
tflite_model = converter.convert()
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)   

  

In [ ]:
tflite_model_file = 'tfmodel/facialbeauty.tflite'
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]      
nb_test_samples = X_ts.shape[0]
nb_rows, nb_cols = 5, 5

**inference by tflite model**

In [ ]:
tflite_model_file = 'tfmodel/facialbeauty.tflite'
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]      
nb_test_samples = X_ts.shape[0]
nb_rows, nb_cols = 5, 5
    
preds=[]
idxs=[]
for k in range(nb_rows * nb_cols):
    i = randint(0, nb_test_samples - 1)
    idxs.append(i)
    interpreter.set_tensor(input_index, np.expand_dims(preprocess_input(X_ts[i].copy()), axis=0))
    interpreter.invoke()
    pred = interpreter.get_tensor(output_index)
    preds.append(pred.flatten()[0])

preds, idxs = zip(*sorted(zip(preds, idxs), reverse=True))

plt.figure(figsize=(10,10))
for k in range(nb_rows * nb_cols):
    plt.subplot(nb_rows, nb_cols, k+1)
    plt.imshow(X_ts[idxs[k]].astype(np.uint8))
    plt.title("p:%.2f" % (preds[k]))
    plt.axis('off')    
plt.tight_layout()
plt.show()
    


In [ ]:
X_new = data.test_data

tflite_model_file = 'tfmodel/facialbeauty.tflite'
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]      
nb_test_samples = X_new.shape[0]
nb_rows, nb_cols = 6, 6
preds=[]
for k in range(nb_rows * nb_cols):
    if k>=X_new.shape[0]:
        break
    interpreter.set_tensor(input_index, np.expand_dims(preprocess_input(X_new[k].copy()), axis=0))
    interpreter.invoke()
    pred = interpreter.get_tensor(output_index)
    preds.append(pred.flatten()[0])

preds, idxs = zip(*sorted(zip(preds, list(range(len(preds)))), reverse=True))

plt.figure(figsize=(15,15))
for k in range(nb_rows * nb_cols):
    if k>=X_new.shape[0]:
        break    
    plt.subplot(nb_rows, nb_cols, k+1)
    plt.imshow(X_new[idxs[k]].astype(np.uint8))
    plt.title("beauty score:%.2f" % (preds[k]))
    plt.axis('off')




In [ ]:
X_new = data.test_data